**Создадим рекомендательную систему, предлагающую пользователям потенциально интересные им посты**

***Шаг 1. Сформируем таблицу, на основе которой будет оцениваться модель***

Проведем EDA (Exploratory Data Analysis). Для этого импортируем нужную библиотеку и откроем данные

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [ ]:

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


user_data = pd.read_sql('SELECT * FROM user_data', con=engine)
post_text_df = pd.read_sql('SELECT * FROM post_text_df', con=engine)
# Третий датасет очень велик, поэтому откроем только первые 10 000 строк для предварительного изучения
feed_data = pd.read_sql('  SELECT * FROM feed_data LIMIT 10000', con=engine)


Взглянем на таблицу user_data


In [ ]:
user_data.head(5)

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


Узнаем, какие значения принимают некоторые стобцы

In [ ]:
age_min = user_data['age'].min()
age_max = user_data['age'].max()

print(f"Minimum age: {age_min}")
print(f"Maximum age: {age_max}")

country_numb = user_data['country'].nunique()
print(f"Number of countries: {country_numb}")

unique_countries = user_data['country'].unique()
print(f"Countries: {unique_countries}")

city_numb = user_data['city'].nunique()
print(f"Number of cities: {city_numb}")

group_numb = user_data['exp_group'].nunique()
print(f"Number of groups: {group_numb}")

unique_os = user_data['os'].unique()
print(f"OS: {unique_os}")

unique_sources = user_data['source'].unique()
print(f"Sources: {unique_sources}")


Minimum age: 14
Maximum age: 95
Number of countries: 11
Countries: ['Russia' 'Ukraine' 'Belarus' 'Azerbaijan' 'Kazakhstan' 'Finland' 'Turkey'
 'Latvia' 'Cyprus' 'Switzerland' 'Estonia']
Number of cities: 3915
Number of groups: 5
OS: ['Android' 'iOS']
Sources: ['ads' 'organic']


Узнаем число пропусков в таблице с данными о пользователях user_data

In [ ]:
user_data.isnull().sum()

,0
user_id,0
gender,0
age,0
country,0
city,0
exp_group,0
os,0
source,0


Взглянем на таблицу user_data

In [ ]:
post_text_df.head(5)

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


Выясним, на какие темы имеют место посты

In [ ]:
topics = post_text_df['topic'].unique()
print(f"Topics: {topics}")

Topics: ['business' 'covid' 'entertainment' 'sport' 'politics' 'tech' 'movie']


Узнаем число пропусков в таблице с постами post_text_df

In [ ]:
post_text_df.isnull().sum()

,0
post_id,0
text,0
topic,0


Ну и посмотрит на последнюю, третью таблицу feed_data, которая содержит данные о действиях

In [ ]:
feed_data.head(12)

,timestamp,user_id,post_id,action,target
0,2021-10-01 22:02:37,2501,5627,view,0
1,2021-10-01 22:05:04,2501,526,view,1
2,2021-10-01 22:05:52,2501,526,like,0
3,2021-10-01 22:05:54,2501,1382,view,0
4,2021-10-01 22:06:36,2501,1423,view,0
5,2021-10-01 22:08:29,2501,1368,view,0
6,2021-10-01 22:10:59,2501,1353,view,0
7,2021-10-01 22:12:41,2501,1678,view,0
8,2021-10-01 22:14:01,2501,1848,view,0
9,2021-10-01 22:14:54,2501,1722,view,0


In [ ]:
topics = feed_data['action'].unique()
print(f"Action: {topics}")

target = feed_data['target'].unique()
print(f"Target: {target}")

Action: ['view' 'like']
Target: [0 1]


*Таким образом, переменная target равна 1, если сутью действия был лайк*

В таблице post_text_df имеется столбец text, который важно преобразовать в ряд численных столбцов, чтобы на их основе потом получить оценки. Ниже это делается в помощью Tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Открываем таблицу с постами
post = pd.read_sql('SELECT post_id, topic, text FROM post_text_df', con=engine)

# Делаем токенизацию текста
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(post['text'])
sequences = tokenizer.texts_to_sequences(post['text'])
padded = pad_sequences(sequences, maxlen=50)

# Добавляем новое измерение
padded = np.expand_dims(padded, axis=-1)

#GRU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense

# Определяем теперь модели
input_layer = Input(shape=(50, 1))
gru_layer = GRU(10, return_sequences=False)(input_layer)
encoder = Model(input_layer, gru_layer)

# Теперь переходим к текстовым векторам
text_vectors = encoder.predict(padded)

text_features = pd.DataFrame(text_vectors,
                           columns=[f'text_feat_{i}' for i in range(10)])

# Объединяем текстовую колонку с числовыми признаками
final_df = pd.concat([post, text_features], axis=1)

220/220 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


Сделаем one hot encoding для колонки topic. Как мы выяснило, она принимает не так много значенией, поэтому one hot encoding вполне подойдет

In [ ]:
one_hot_columns = [ 'topic' ]

for col in one_hot_columns:
    one_hot = pd.get_dummies(final_df[col], prefix=col, drop_first=True)
    final_df = pd.concat((final_df, one_hot), axis=1)

Запишем полученную таблицу, содержащую данные о постах, в базу данных SQL

In [ ]:
final_df.to_sql('post_final_2gru', con=engine, index = False, if_exists='replace') # записываем таблицу

# И ее короткую версию, где только числовые столбцы, тоже
a = final_df.drop(columns=['text', 'topic'])
a.to_sql('post_final_2gru_short', con=engine, index = False, if_exists='replace') # записываем таблицу

23

Сформируем единую таблицу, которая объединит все три изученные выше таблицы

In [ ]:
# Откроем таблицы
user_data = pd.read_sql('SELECT * FROM user_data', con=engine)
post_final_2gru = pd.read_sql('SELECT * FROM post_final_2gru', con=engine)
# Из третьего датасета возьмем только часть строк, так как целиком он слишком велик
feed_data = pd.read_sql('  SELECT timestamp, user_id, post_id, action, target FROM feed_data LIMIT 500000 ', con=engine)

In [ ]:
# А теперь непосредственно объединение
combined = pd.merge(feed_data, user_data, on='user_id',how='left')
combined = pd.merge(combined, post_final_2gru, on='post_id',how='left')

In [ ]:
combined = combined.drop(columns=['text'])

In [ ]:
combined = combined.drop(columns=['topic'])

Отсортируем базу данных по времени, чтобы впоследствии валидация происходила хронологически, то есть на основе "старых" данных предсказывались более "новые" действия пользователей

In [ ]:
combined = combined.sort_values(by='timestamp')

*Теперь дополним датасет новыми переменными и скорректируем его. Сделаем его более содержательным и оптимальным для целей получения оценок*

Предполагается, что вкусы и интересы различаются по возрастным группам. Подростки (до 17 лет) учатся в школе и имеют соответствующие интересы. Люди "студенческого" возраста заметно отличаются от первых в силу изменения социального статуса и образа жизни. Что касается людей старше 25 лет, то это в полной мере повзрослевшие пользователи; при этом различия внутри этой группы уже сложнее поставить в зависимость от возраста

In [ ]:
# Cоздаем дамми-переменные на возраста
combined['age_below_17'] = (combined['age'] < 17).astype(int)
combined['age_18_25'] = ((combined['age'] >= 18) & (combined['age'] <= 25)).astype(int)
# Возраст свыше 25 уже как третий оставшийся вариант не добавляем, естественно
combined = combined.drop(columns=['age'])

Присутствуют текстовые признаки, которые требуют one hot преобразования

In [ ]:
one_hot_columns = [ 'exp_group', 'os', 'source']

for col in one_hot_columns:
    one_hot = pd.get_dummies(combined[col], prefix=col, drop_first=True)
    combined = pd.concat((combined.drop(col, axis=1), one_hot), axis=1)

Текстовые признаки, принимающие большое количество уникальных значений, преобразуем по методу mean target encoding

In [ ]:
# Проведем эту процедуру для столбца country
mean_target = combined.groupby('country')['target'].mean()

# Присваиваем средние значения  обратно в DataFrame
combined['country_mean_target'] = combined['country'].map(mean_target)
combined = combined.drop(columns=['country'])

# Проведем эту процедуру для столбца city
mean_target = combined.groupby('city')['target'].mean()

# Присваиваем средние значения  обратно в DataFrame
combined['city_mean_target'] = combined['city'].map(mean_target)
combined = combined.drop(columns=['city'])

Удалим строки, где признак action принимает значение like, так как признак target и так свидетельствует о том, какое действие произвел пользователь

In [ ]:
combined = combined[combined['action'] != 'like']
# Теперь признка action уже не нужен
combined = combined.drop(columns=['action'])

Поскольку мы уже отсортировали по timestamp , то можно удалить колонку, тем не менее валидируясь хронологически

In [ ]:
combined = combined.drop(columns=['timestamp'])

Выделим матрицу признаков и вектор целевой переменной

In [ ]:
X = combined.drop(columns=['target'])
Y = combined['target']

Выделим обучающую и тестовую выборки

In [ ]:
# Возьмем соотношение 4 к 1
train_size = int(len(combined) * 4/5)

# Выделим обучающую часть
X_train = X.iloc[:train_size]
Y_train = Y.iloc[:train_size]

# Выделим тестовую часть
X_test = X.iloc[train_size:]
Y_test = Y.iloc[train_size:]

***Шаг 2.Обучим модель***

Путем экспериментального сравнения было решено использовать XGBoost

In [ ]:
from xgboost import XGBClassifier

# Пайплайн с XGBoost
pipe = Pipeline([
    ('scaler', StandardScaler()),  # масштабирование не всегда нужно для XGBoost
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss'))
])

pipe.fit(X_train, Y_train)

# Делаем предсказание
y_pred_test = pipe.predict(X_test)

# Оцениваем accuracy
accuracy_test = accuracy_score(Y_test, y_pred_test)
# Выводим результат
print("Accuracy на тестовой выборке:", accuracy_test)

Accuracy на тестовой выборке: 0.8616595921018737


Подберем наилучшие параметры

In [ ]:
from sklearn.model_selection import GridSearchCV

# Набор параметров
param_grid = {
    'xgb__max_depth': [2, 3, 4],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
}

# Инициализация GridSearchCV
grid_search2 = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=0)

# Обучение модели с использованием GridSearchCV
grid_search2.fit(X_train, Y_train)

# Получение лучших параметров
best_params2 = grid_search2.best_params_
print("Лучшие параметры:", best_params)
print("Лучшая точность:", grid_search2.best_score_)

Лучшие параметры: {'LR__C': 0.01, 'LR__penalty': 'l2'}
Лучшая точность: 0.884567569387998


In [ ]:
# Создаем новый пайплайн с лучшими параметрами
best_pipe2 = grid_search2.best_estimator_

# Переобучаем на всех тренировочных данных
best_pipe2.fit(X_train, Y_train)

# Оценка качества на случай, если изменилось что-то в таблице, а предыдущая ячейка не воспроизводилась
y_pred = best_pipe2.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, y_pred))

Accuracy: 0.8617603906503747


In [ ]:
import pickle

filename = 'sklearn_model_x_post_XG.pkl'
pickle.dump(pipe, open(filename, 'wb'))


***Шаг 3. Создадим таблицу с теми же столбцами, что и в таблице combined. В ней будут присутствовать все user_id. На ее основе каждому пользователю будут предлагаться потенциально подходящие посты***

In [ ]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


post_final_2gru = pd.read_sql('SELECT * FROM post_final_2gru', con=engine)

# Объединяем таблицы с помощью запроса SQL
combined2 = pd.read_sql('  SELECT * FROM user_data LEFT JOIN (SELECT * FROM feed_data LIMIT 300000) AS feed_data_short using(user_id)  ', con=engine)
combined2 = pd.merge(combined2, post_final_2gru, on='post_id',how='left')

# Проделываем те же операции по дорабатыванию таблицы, что и те, которые были проделаны выше для таблицы combined
combined2['age_below_17'] = (combined2['age'] < 17).astype(int)
combined2['age_18_25'] = ((combined2['age'] >= 18) & (combined2['age'] <= 25)).astype(int)
combined2 = combined2.drop(columns=['age'])

combined2 = combined2.drop(columns=['topic'])

one_hot_columns = [ 'exp_group', 'os', 'source' ]

for col in one_hot_columns:
    one_hot = pd.get_dummies(combined2[col], prefix=col, drop_first=True)
    combined2 = pd.concat((combined2.drop(col, axis=1), one_hot), axis=1)

mean_target = combined2.groupby('city')['target'].mean()

combined2['city_mean_target'] = combined2['city'].map(mean_target)
combined2 = combined2.drop(columns=['city'])

mean_target = combined2.groupby('country')['target'].mean()

combined2['country_mean_target'] = combined2['country'].map(mean_target)
combined2 = combined2.drop(columns=['country'])

combined2 = combined2[combined2['action'] != 'like']
combined2 = combined2.drop(columns=['action'])

combined2 = combined2.sort_values(by='timestamp')
combined2 = combined2.drop(columns=['timestamp'])

for column in combined2.columns:
    mode_value = combined2[column].mode()[0]  # Получаем самое популярное значение
    combined2[column].fillna(mode_value, inplace=True)

combined2 = combined2[X.columns]


Загрузим полученную таблицу в базу данных

In [ ]:
combined2.to_sql('aleksandr_tomaev_features', con=engine, index=False, if_exists='replace')


347

Также загрузим в базу данных укороченную версию последней таблицы. Она пригодится при написании кода для сервиса

In [ ]:
# Столбцы, которые не должны присутствовать в укороченной таблице (описания постов)
columns_to_drop = [
'post_id',
'text_feat_0',
'text_feat_1',
'text_feat_2',
'text_feat_3',
'text_feat_4',
'text_feat_5',
'text_feat_6',
'text_feat_7',
'text_feat_8',
'text_feat_9',
'topic_covid',
'topic_entertainment',
'topic_movie',
'topic_politics',
'topic_sport',
'topic_tech'
]

In [ ]:
# Удаляем эти столбцы
short = combined2.drop(columns=columns_to_drop)
# Удаляем дубликаты
short = short.drop_duplicates(subset=['user_id'])


Загружаем в базу данных

In [ ]:
short.to_sql('aleksandr_tomaev_features_short', con=engine, index=False, if_exists='replace')


205

***Шаг 4. Далее на основе сформированных таблиц и сохраненной модели пользователям будут рекомендоваться посты***

Сам код представлен в отдельном файле my_service_1